In [1]:
import torch
from torch import nn
from torch.nn import functional as F

In [21]:
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand(size=(2, 4))
net(X)

tensor([[0.0490],
        [0.1621]], grad_fn=<AddmmBackward0>)

In [17]:
# 访问目标参数
print(net[2].state_dict())
print(net[2].weight)
print(net[2].weight.data)
print(net[2].weight.grad == None) # 尚未计算，初始梯度为None

OrderedDict([('weight', tensor([[-0.3498,  0.0376,  0.0554, -0.2814,  0.0986, -0.2722,  0.1307,  0.2526]])), ('bias', tensor([-0.1345]))])
Parameter containing:
tensor([[-0.3498,  0.0376,  0.0554, -0.2814,  0.0986, -0.2722,  0.1307,  0.2526]],
       requires_grad=True)
tensor([[-0.3498,  0.0376,  0.0554, -0.2814,  0.0986, -0.2722,  0.1307,  0.2526]])
True


In [22]:
# 获得所有参数
print(*[(name, param.shape) for name, param in net[0].named_parameters()])

print(*[(name, param.shape) for name, param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [23]:
# 从嵌套块收集参数
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8,4), nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in range(4):
        net.add_module(f'block {i}', block1())
    return net

rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)

tensor([[0.0331],
        [0.0331]], grad_fn=<AddmmBackward0>)

In [24]:
# 观察嵌套块

In [25]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


In [27]:
rgnet[0][1][0].bias.data

tensor([ 0.3581, -0.3568, -0.2340, -0.4224,  0.1499, -0.3376, -0.2902, -0.2701])

In [28]:
# 参数初始化
def init_weight(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, 0, 0.01)
        nn.init.zeros_(m.bias)

net.apply(init_weight)
net[0].weight.data[0],net[0].bias.data[0]

(tensor([-0.0100,  0.0033, -0.0161, -0.0075]), tensor(0.))

In [30]:
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)
net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

In [31]:
def init_xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)
def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)

net[0].apply(init_xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([-0.1713,  0.0393,  0.7033,  0.7048])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


In [42]:
# 自定义初始化
def my_init(m):
    if type(m) == nn.Linear:
        print("Init", *[(name, param.shape) for name, param in m.named_parameters()][0])
        nn.init.uniform_(m.weight, -10, 10)
        m.weight.data *= m.weight.data.abs() >=5
    
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(),nn.Linear(8, 1))
net.apply(my_init)
net[0].weight[:2]

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[-0.0000,  7.4474, -0.0000, -0.0000],
        [ 0.0000,  5.1335, -7.4634,  0.0000]], grad_fn=<SliceBackward0>)

In [43]:
net[0].weight.data[:] += 1
net[0].weight.data[0, 0] = 42
net[0].weight.data[0]

tensor([42.0000,  8.4474,  1.0000,  1.0000])

In [45]:
# 参数绑定
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                    shared, nn.ReLU(),
                    shared, nn.ReLU(),
                    nn.Linear(8, 1))
X = torch.randn(10, 4)
net(X)
# 检查参数是否相同
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100 # 修改其中一个对象，以验证两者实际上是同一对象而非仅值相同
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])
